In [2]:
# Library imports
import pyforest
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
from tqdm import tqdm
from pprint import pprint
from time import sleep
import time
import seaborn as sns

from turtle import forward
import torch.nn as nn
import torch.nn.functional as F
import math
import torch
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from torch.utils.data import DataLoader, TensorDataset

#### **Hyperparams and data-loading**

In [3]:
train_edges = np.load('movielens/train_edges.npy')
user_list = train_edges[:, 0]
item_list = train_edges[:, 1]
rating_list = train_edges[:, 2].astype('float32')

n_users = user_list.max() + 1 
n_items = item_list.max() + 1
n_samples = len(rating_list)

n_users, n_items, n_samples

(943, 1682, 200000)

#### **Neural collaborative filtering**

In [33]:
class NCF(Module):
    def __init__(self, n_users, n_items, n_factors):
        super(NCF, self).__init__()
        self.user_emb = nn.Embedding(n_users, n_factors)
        self.item_emb = nn.Embedding(n_items, n_factors)
        self.fc = nn.Linear(n_factors * 2, 1)
        # self.user_emb.weight.data.uniform_(0, 0.5)
        # self.item_emb.weight.data.uniform_(0, 0.5)
        # self.fc.weight.data.uniform_(0, 0.5)

    def forward(self, user, item):
        u = self.user_emb(user)
        i = self.item_emb(item)
        features = torch.concat([u, i], dim = 1)
        x = self.fc(features)
        out = torch.sigmoid(x)
        return out

In [8]:
class NCF(Module):
    def __init__(self, n_users, n_items, n_factors):
        super(NCF, self).__init__()
        self.user_emb = nn.Embedding(n_users, n_factors)
        self.item_emb = nn.Embedding(n_items, n_factors)
        self.fc1 = nn.Linear(n_factors * 2, n_factors)
        # self.fc2 = nn.Linear(n_factors, n_factors // 2)
        self.fc2 = nn.Linear(n_factors, 1)
        # self.fc3 = nn.Linear(n_factors // 2, 1)

    def forward(self, user, item):
        tanh = nn.Tanh()
        sigmoid = nn.Sigmoid()

        u = self.user_emb(user)
        i = self.item_emb(item)
        features = torch.concat([u, i], dim = 1)
        x = self.fc1(features)
        x = tanh(x)
        x = self.fc2(x)
        # x = tanh(x)
        # x = self.fc3(x)
        x = sigmoid(x)
        return x

In [5]:
class CollaborativeFiltering(Module):
    def __init__(self, n_users, n_items, n_factors):
        super(CollaborativeFiltering, self).__init__()
        self.user_emb = nn.Embedding(n_users, n_factors)
        self.item_emb = nn.Embedding(n_items, n_factors)

    def forward(self, user, item):
        u = self.user_emb(user)
        i = self.item_emb(item)
        dot = (u * i).sum(1)
        return torch.sigmoid(dot)

In [6]:
def get_accuracy(y_hat, y):
    y = y.clone().int()
    y_hat = (y_hat.clone() > 0.5).int()
    accuracy = (y == y_hat).sum() / len(y)
    return accuracy.item()

#### **Test NCF**

In [44]:
# use gpu 
use_gpu = 6
if use_gpu == -1:
    device = 'cpu'
else:
    device = torch.device('cuda:{}'.format(str(use_gpu)) if torch.cuda.is_available() else 'cpu')

# load users, items and ratings as tensors
users = torch.tensor(user_list, device = device)
items = torch.tensor(item_list, device = device)
ratings = torch.tensor(rating_list, device = device)
ratings = ratings.reshape((len(ratings), 1))

# define model and it's parameters
n_factors = 64
lr = 65
T = 501

for lr in [1, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500, 1000]:

    for T in [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000]:

        model = NCF(n_users, n_items, n_factors)
        # model = CollaborativeFiltering(n_users, n_items, n_factors)
        model.to(device)

        manual_gradients = False
        if manual_gradients == True:
            p1, p2 = model.parameters()
        else:
            optimizer = torch.optim.SGD(model.parameters(), lr = lr)

        torch.manual_seed(0)
        for layer in model.children():
            layer.reset_parameters()

        loss_fn = nn.BCELoss(reduction = 'mean')
        model.train()

        if manual_gradients == True:
            for i in range(T):
                y_hat = model(users, items)
                loss = loss_fn(y_hat, ratings)
                # results.append([delta, i, loss.item()])
                if i % 10 == 0:
                    print('T = ', T, ' loss: ', loss.item(), ' acc: ', get_accuracy(y_hat, ratings))

                # compute inner parameter gradients
                p1_grad = torch.autograd.grad(loss, p1, retain_graph=retain_graph, create_graph=create_graph)
                p2_grad = torch.autograd.grad(loss, p2, retain_graph=retain_graph, create_graph=create_graph)

                # update inner parameters
                with torch.no_grad():
                    p1_new = p1 - lr * p1_grad[0]
                    p2_new = p2 - lr * p2_grad[0]
                    p1.copy_(p1_new)
                    p2.copy_(p2_new)
        else:
            for i in range(T):
                y_hat = model(users, items)
                loss = loss_fn(y_hat, ratings)
                # results.append([delta, i, loss.item()])
                if (i % (T - 1) == 0) and  (i > 0):
                    print('lr: ', lr, ' T = ', i + 1, ' loss: ', loss.item(), ' acc: ', get_accuracy(y_hat, ratings))
                    

                # use torch.optim optimizer to compute gradients
                optimizer.zero_grad()
                loss.backward(retain_graph=True, create_graph=False)
                optimizer.step()
    print()

lr:  1  T =  50  loss:  0.6828802824020386  acc:  0.5540849566459656
lr:  1  T =  100  loss:  0.6806814074516296  acc:  0.5611050128936768
lr:  1  T =  150  loss:  0.6764176487922668  acc:  0.5745750069618225
lr:  1  T =  200  loss:  0.6643857359886169  acc:  0.6038050055503845
lr:  1  T =  250  loss:  0.6322505474090576  acc:  0.6566149592399597
lr:  1  T =  300  loss:  0.5861669182777405  acc:  0.7042199969291687
lr:  1  T =  350  loss:  0.5473552942276001  acc:  0.7322149872779846
lr:  1  T =  400  loss:  0.5206984281539917  acc:  0.7523349523544312
lr:  1  T =  450  loss:  0.5298035144805908  acc:  0.7421799898147583
lr:  1  T =  500  loss:  0.49649155139923096  acc:  0.7652599811553955
lr:  1  T =  550  loss:  0.49068909883499146  acc:  0.7680400013923645
lr:  1  T =  600  loss:  0.48390406370162964  acc:  0.7710899710655212
lr:  1  T =  650  loss:  0.47969263792037964  acc:  0.7729150056838989
lr:  1  T =  700  loss:  0.4759491980075836  acc:  0.7744999527931213
lr:  1  T =  750 

In [50]:
u = torch.tensor([1,2,3,4])
v = torch.tensor([5,6,7,8])
torch.concat([u, v], dim = 1)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [ ]:
user_emb = nn.Embedding(n_users, n_factors)
item_emb = nn.Embedding(n_items, n_factors)
